In [5]:
# import boto3
# import sagemaker
import pandas as pd
# from pyathena import connect

#### Auth with AWS

In [3]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

#### Convert csv to tsv and move to S3 

In [22]:
s3_private_data_path = "s3://{}/w2-musicData/csv".format(bucket)
print(s3_private_data_path)

s3://sagemaker-us-east-1-106006112223/w2-musicData/csv


In [7]:
org_df = pd.read_csv('dataset.csv')
display(org_df.head())

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [9]:
org_df = org_df.drop(columns=['album_name'])
display(org_df.head())

,Unnamed: 0,track_id,artists,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [11]:
org_df.to_csv('dataset_clean.csv', index=False)  

In [23]:
!aws s3 cp "dataset_clean.csv" $s3_private_data_path/

upload: ./dataset_clean.csv to s3://sagemaker-us-east-1-106006112223/w2-musicData/csv/dataset_clean.csv


In [24]:
!aws s3 ls $s3_private_data_path/

2024-09-16 23:32:03   17618837 dataset_clean.csv


#### Create DB in Athena for queries

In [7]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
database_name = "w2_music_db"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
print(s3_staging_dir)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

s3://sagemaker-us-east-1-106006112223/athena/staging


In [22]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
pd.read_sql(statement, conn)

statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_554/2702660478.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


#### Create tables in DB and schemas

In [29]:
table_name_tsv = 'music_ds_tsv6'
database_name = "w2_music_db"
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         track_id string,
         artists string,
         track_name string,
         popularity int,
         duration_ms int,
         explicit boolean,
         danceability int,
         energy int,
         key int,
         loudness int,
         mode int,
         speechiness float,
         acousticness float,
         instrumentalness float,
         liveness float,
         valence float,
         tempo float,
         time_signature int,
         track_genre string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\\t' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, table_name_tsv,s3_private_data_path
)

print(statement)

pd.read_sql(statement, conn)

CREATE EXTERNAL TABLE IF NOT EXISTS w2_music_db.music_ds_tsv6(
         track_id string,
         artists string,
         track_name string,
         popularity int,
         duration_ms int,
         explicit boolean,
         danceability int,
         energy int,
         key int,
         loudness int,
         mode int,
         speechiness float,
         acousticness float,
         instrumentalness float,
         liveness float,
         valence float,
         tempo float,
         time_signature int,
         track_genre string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-106006112223/w2-musicData/csv'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


/tmp/ipykernel_253/3576572253.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [30]:
statement = "SHOW TABLES IN W2_MUSIC_DB"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_253/1294112312.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,music_ds_tsv6


In [ ]:
# first test query
category = "Bad Bunny"

statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, table_name_tsv
)
print(statement)
sql_df = pd.read_sql(statement, conn)
display(sql_df.head(5))

In [15]:
#calculate number of columns in CSV file
with open('dataset_clean.csv') as x:
    ncols = len(x.readline().split(','))


full_df = pd.read_csv('dataset_clean.csv', usecols=range(1,ncols))
display(full_df.head())
display(full_df.tail())


,track_id,artists,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


,track_id,artists,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,Sleep My Little Boy,21,384999,False,0.172,0.235,5,-16.393,1,0.0422,0.640,0.928,0.0863,0.0339,125.995,5,world-music
113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,Water Into Light,22,385000,False,0.174,0.117,0,-18.318,0,0.0401,0.994,0.976,0.1050,0.0350,85.239,4,world-music
113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Miss Perfumado,22,271466,False,0.629,0.329,0,-10.895,0,0.0420,0.867,0.000,0.0839,0.7430,132.378,4,world-music
113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Friends,41,283893,False,0.587,0.506,7,-10.889,1,0.0297,0.381,0.000,0.2700,0.4130,135.960,4,world-music
113999,2hETkH7cOfqmz3LqZDHZf5,Cesária Evora,Barbincor,22,241826,False,0.526,0.487,1,-10.204,0,0.0725,0.681,0.000,0.0893,0.7080,79.198,4,world-music


### Homework queries

#### 1. List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99

In [12]:
statement = """SELECT artists, track_name, popularity FROM {}.{}
    WHERE popularity >= 99 LIMIT 10""".format(
    database_name, table_name_tsv
)
print(statement)

df = pd.read_sql(statement, conn)
df.head(10)

SELECT artists, track_name, popularity FROM w2_music_db.music_ds_tsv4
    WHERE popularity >= 99 LIMIT 10


/tmp/ipykernel_253/944662429.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


DatabaseError: Execution failed on sql: SELECT artists, track_name, popularity FROM w2_music_db.music_ds_tsv4
    WHERE popularity >= 99 LIMIT 10
TYPE_MISMATCH: line 2:22: Cannot apply operator: varchar <= integer
unable to rollback

In [19]:
# pandas
pd_df = full_df[full_df['popularity'] >= 99][['artists','track_name','popularity']]
display(pd_df.head())

,artists,track_name,popularity
20001,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100
51664,Bizarrap;Quevedo,"Quevedo: Bzrp Music Sessions, Vol. 52",99
81051,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100


#### 2. List artists with an average popularity of 92


In [ ]:
# SELECT artists
# FROM w2_music_db
# GROUP BY artists
# HAVING AVG(popularity) = 92;


statement = """SELECT artists FROM {}.{}
    GROUP BY artists HAVING AVG(popularity) = 92""".format(
    database_name, table_name_tsv
)

print(statement)

df = pd.read_sql(statement, conn)
df.head(10)

In [21]:
# pandas
artists_avg_popularity = full_df.groupby('artists').filter(lambda x: x['popularity'].mean() == 92)
display(artists_avg_popularity.head())
artists_avg_popularity_list = artists_avg_popularity['artists'].unique()
print(artists_avg_popularity_list)


,track_id,artists,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
81052,4LRPiXqCikLlN15c3yImP7,Harry Styles,As It Was,95,167303,False,0.520,0.731,6,-5.338,0,0.0557,0.342,0.001010,0.311,0.662,173.930,4,pop
81100,0WtM2NBVQNNJLh6scP13H8,Rema;Selena Gomez,Calm Down (with Selena Gomez),92,239317,False,0.801,0.806,11,-5.206,1,0.0381,0.382,0.000669,0.114,0.802,106.999,4,pop
81158,6UelLqGlWMcVH1E5c4H7lY,Harry Styles,Watermelon Sugar,89,174000,False,0.548,0.816,0,-4.209,1,0.0465,0.122,0.000000,0.335,0.557,95.390,4,pop
81205,4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,92,167303,False,0.520,0.731,6,-5.338,0,0.0557,0.342,0.001010,0.311,0.662,173.930,4,pop


['Harry Styles' 'Rema;Selena Gomez']


#### 3. List the Top 10 most energetic genres
  

In [ ]:
# SELECT track_genre, AVG(energy) AS avg_energy
# FROM w2_music_db.tracks
# GROUP BY track_genre
# ORDER BY avg_energy DESC
# LIMIT 10;

statement = """SELECT track_genre, AVG(energy) AS avg_energy FROM {}.{}
    GROUP BY track_genre 
    ORDER BY avg_energy DESC LIMIT 10""".format(
    database_name, table_name_tsv
)

print(statement)
df = pd.read_sql(statement, conn)
df.head(10)

In [23]:
# pandas
top_energetic_genres = full_df.groupby('track_genre')['energy'].mean().sort_values(ascending=False).head(10)
display(top_energetic_genres)

track_genre
death-metal      0.931470
grindcore        0.924201
metalcore        0.914485
happy            0.910971
hardstyle        0.901246
drum-and-bass    0.876635
black-metal      0.874897
heavy-metal      0.874003
party            0.871237
j-idol           0.868677
Name: energy, dtype: float64

#### 4. How many tracks is Bad Bunny on?

In [ ]:
# SELECT COUNT(*) AS track_count
# FROM w2_music_db.tracks
# WHERE artists LIKE '%Bad Bunny%';

statement = """SELECT COUNT(*) AS track_count FROM {}.{}
    WHERE artists LIKE '%Bad Bunny%'""".format(
    database_name, table_name_tsv
)

print(statement)
df = pd.read_sql(statement, conn)
print(df)

In [27]:
full_df = full_df.dropna()
bad_bunny_tracks_count = full_df[full_df['artists'].str.contains('Bad Bunny')].shape[0]
print(bad_bunny_tracks_count)

416


#### 5. Show the top 10 genres in terms of popularity sorted by their most popular track

In [ ]:
# SELECT track_genre, MAX(popularity) AS max_popularity
# FROM w2_music_db.tracks
# GROUP BY track_genre
# ORDER BY max_popularity DESC
# LIMIT 10;

statement = """SELECT track_genre, MAX(popularity) AS max_popularity FROM {}.{}
    GROUP BY track_genre
    ORDER BY max_popularity DESC
    LIMIT 10""".format(
    database_name, table_name_tsv
)

print(statement)
df = pd.read_sql(statement, conn)
df.head(10)

In [28]:

# pandas
top_genres_by_popularity = full_df.groupby('track_genre')['popularity'].max().sort_values(ascending=False).head(10)
print(top_genres_by_popularity)

track_genre
dance        100
pop          100
hip-hop       99
latin         98
edm           98
latino        98
reggaeton     98
reggae        98
rock          96
piano         96
Name: popularity, dtype: int64


In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}